In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import numpy as np

from pathlib import Path
import os
import zipfile
import matplotlib.pyplot as plt

# Data

In [2]:
# Read dictionary with municipalities
muni = pd.read_csv('/Users/andreza/reproducibility_transportation_hubs-early_warning_surveillance_systems/data/DTB_BRASIL_MUNICIPIO.csv',sep=';')    

    
#'/Users/julianeoliveira/Documents/Projects/AESOP/AESOP datalake/

In [38]:
# Read the Adjacent matrix
link0 = '/Users/andreza/reproducibility_transportation_hubs-early_warning_surveillance_systems/code/adjacency_matrix_complete_4.parquet'

#'/home/juliane.oliveira/Documents/AESOP data structure/ETL_data/adjacency_matrix_final.parquet'

#/Users/julianeoliveira/Documents/Projects/AESOP/AESOP datalake/Mobility - Road-air-fluvial networks/adjacency_matrix.parquet'

matrix = pd.read_parquet(link0, engine='pyarrow')

# Functions

In [39]:
link_muni_vertice = pd.DataFrame(matrix.columns, columns=['muni'])

In [40]:
def get_mname(n):
    
    m = link_muni_vertice.iloc[n]['muni']
    set_muni = muni[muni['Código Município Completo'] == m].reset_index()
    return [set_muni.iloc[0]['Nome_Município'],set_muni.iloc[0]['Nome_UF'],m]
   
def get_mnumber(name):
    muni[muni['Nome_Município'] == name]
    
    co_mu = muni[muni['Nome_Município'] == name].reset_index()['Código Município Completo'][0]
    muni_number = link_muni_vertice[link_muni_vertice['muni'] == co_mu]['muni'].index.tolist()[0]
    return [muni_number, co_mu]

In [41]:
df_np = matrix.to_numpy()

# Paths

In [42]:
# Read the data
data_dir = Path('/Users/andreza/Downloads/FF_path_results_Manaus_4/')


                
#full_df = pd.concat(
#    pd.read_parquet(parquet_file)
#    for parquet_file in data_dir.glob('*.parquet')
#)

In [43]:
lis_dfs = []
for parquet_file in data_dir.glob('*.parquet'):
    df = pd.read_parquet(parquet_file)
    #df = df.reset_index()
    #df['value'].iloc[0] = df['value_cum'].iloc[0]
    lis_dfs.append(df)

In [44]:
len(lis_dfs) #26043  2484

1390

In [45]:
for dfs in lis_dfs:
    if len(dfs) != 0:  dfs['value'].iloc[0] = dfs['value_cum'].iloc[0]
    else:
        pass
    


/Users/andreza/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [46]:
lis_dfs2 = []

for df in lis_dfs:
    
    new_df = df[df['value'] == df['value'].max()]
    
    lis_dfs2.append(new_df)

In [47]:
len(lis_dfs2)

1390

In [48]:
df = pd.concat(lis_dfs2)
df = df.reset_index()

In [49]:
len(df)

1452

In [50]:
df.head()

,index,value_cum,value,ori_muni_name,ori_uf_name,ori_co_ibge,des_muni_name,des_uf_name,des_co_ibge,paths
0,9,123.00,42.00,Manaus,Amazonas,1302603,Areias,São Paulo,3503505,"[[3415, 3307], [3829, 3415], [111, 3829], None]"
1,3,34.00,14.00,Manaus,Amazonas,1302603,Careiro,Amazonas,1301100,"[[110, 91], [111, 110]]"
2,11,176.96,47.18,Manaus,Amazonas,1302603,Além Paraíba,Minas Gerais,3101508,"[[3242, 2258], [1596, 3242], [111, 1596], None]"
3,0,223.88,223.88,Manaus,Amazonas,1302603,Peruíbe,São Paulo,3537602,"[[3829, 3690], [111, 3829], None, None]"
4,0,63.00,63.00,Manaus,Amazonas,1302603,Caxias,Maranhão,2103000,"[[169, 501], [111, 169], None, None]"


In [51]:
df.des_uf_name.unique()

array(['São Paulo', 'Amazonas', 'Minas Gerais', 'Maranhão', 'Ceará',
       'Paraná', 'Bahia', 'Alagoas', 'Pará', 'Espírito Santo',
       'Rio Grande do Norte', 'Piauí', 'Pernambuco', 'Rio Grande do Sul',
       'Santa Catarina', 'Paraíba', 'Goiás', 'Rio de Janeiro', 'Sergipe',
       'Tocantins', 'Mato Grosso', 'Mato Grosso do Sul', 'Roraima',
       'Rondônia', 'Acre', 'Amapá', 'Distrito Federal'], dtype=object)

In [53]:
def get_incorret_path(value):
    desired_order = []
    for lst in value:
        if lst is not None:
            lst = list(lst)
            desired_order.append(lst)

    desired_order.reverse()
    
    res = pd.DataFrame(desired_order, columns=['ori','des'])

    res_list = list(res.ori) + list(res.des[-1:])
    
    return res_list


In [54]:
path_correct = []

for value in df['paths']:
    
    path_correct.append(get_incorret_path(value))
  

In [55]:
df = df.assign(path_correct = path_correct)

In [56]:
df.head()

,index,value_cum,value,ori_muni_name,ori_uf_name,ori_co_ibge,des_muni_name,des_uf_name,des_co_ibge,paths,path_correct
0,9,123.00,42.00,Manaus,Amazonas,1302603,Areias,São Paulo,3503505,"[[3415, 3307], [3829, 3415], [111, 3829], None]","[111, 3829, 3415, 3307]"
1,3,34.00,14.00,Manaus,Amazonas,1302603,Careiro,Amazonas,1301100,"[[110, 91], [111, 110]]","[111, 110, 91]"
2,11,176.96,47.18,Manaus,Amazonas,1302603,Além Paraíba,Minas Gerais,3101508,"[[3242, 2258], [1596, 3242], [111, 1596], None]","[111, 1596, 3242, 2258]"
3,0,223.88,223.88,Manaus,Amazonas,1302603,Peruíbe,São Paulo,3537602,"[[3829, 3690], [111, 3829], None, None]","[111, 3829, 3690]"
4,0,63.00,63.00,Manaus,Amazonas,1302603,Caxias,Maranhão,2103000,"[[169, 501], [111, 169], None, None]","[111, 169, 501]"


In [57]:
path_len = []
for value in df.path_correct:
    x = len(value)
    path_len.append(x)

In [58]:
df = df.assign(path_len = path_len)

In [59]:
df = df.assign(path_len = df.path_len -1)

In [60]:
df.head()

,index,value_cum,value,ori_muni_name,ori_uf_name,ori_co_ibge,des_muni_name,des_uf_name,des_co_ibge,paths,path_correct,path_len
0,9,123.00,42.00,Manaus,Amazonas,1302603,Areias,São Paulo,3503505,"[[3415, 3307], [3829, 3415], [111, 3829], None]","[111, 3829, 3415, 3307]",3
1,3,34.00,14.00,Manaus,Amazonas,1302603,Careiro,Amazonas,1301100,"[[110, 91], [111, 110]]","[111, 110, 91]",2
2,11,176.96,47.18,Manaus,Amazonas,1302603,Além Paraíba,Minas Gerais,3101508,"[[3242, 2258], [1596, 3242], [111, 1596], None]","[111, 1596, 3242, 2258]",3
3,0,223.88,223.88,Manaus,Amazonas,1302603,Peruíbe,São Paulo,3537602,"[[3829, 3690], [111, 3829], None, None]","[111, 3829, 3690]",2
4,0,63.00,63.00,Manaus,Amazonas,1302603,Caxias,Maranhão,2103000,"[[169, 501], [111, 169], None, None]","[111, 169, 501]",2


In [61]:
df.path_len.max()

4

In [62]:
def split_path_mun1(col_lst):
    muni_1 = []
    for value in col_lst:
        fac = value[1]
        
        muni_1.append(fac)
        
    return muni_1

def split_path_mun2(col_lst):
    muni_2 = []
    for value in col_lst:
        if len(value) == 2: fac = 'nan'
        elif len(value) >=3: fac = value[2]
        
        muni_2.append(fac)
        
    return muni_2

def split_path_mun3(col_lst):
    muni_3 = []
    for value in col_lst:
        if len(value) == 2: fac = 'nan'
        elif len(value) ==3: fac = 'nan'
        elif len(value) >=4: fac = value[3]
        
        muni_3.append(fac)
        
    return muni_3

In [63]:
#df = df.assign(Rio_branco_AC = check_city_in_path(df.path_correct,66))
df = df.assign(muni_1 = split_path_mun1(df.path_correct))

df = df.assign(muni_2 = split_path_mun2(df.path_correct))

df = df.assign(muni_3 = split_path_mun3(df.path_correct))

In [64]:
df = df.assign(ones = 1)

In [65]:
df.head()

,index,value_cum,value,ori_muni_name,ori_uf_name,ori_co_ibge,des_muni_name,des_uf_name,des_co_ibge,paths,path_correct,path_len,muni_1,muni_2,muni_3,ones
0,9,123.00,42.00,Manaus,Amazonas,1302603,Areias,São Paulo,3503505,"[[3415, 3307], [3829, 3415], [111, 3829], None]","[111, 3829, 3415, 3307]",3,3829,3415,3307,1
1,3,34.00,14.00,Manaus,Amazonas,1302603,Careiro,Amazonas,1301100,"[[110, 91], [111, 110]]","[111, 110, 91]",2,110,91,nan,1
2,11,176.96,47.18,Manaus,Amazonas,1302603,Além Paraíba,Minas Gerais,3101508,"[[3242, 2258], [1596, 3242], [111, 1596], None]","[111, 1596, 3242, 2258]",3,1596,3242,2258,1
3,0,223.88,223.88,Manaus,Amazonas,1302603,Peruíbe,São Paulo,3537602,"[[3829, 3690], [111, 3829], None, None]","[111, 3829, 3690]",2,3829,3690,nan,1
4,0,63.00,63.00,Manaus,Amazonas,1302603,Caxias,Maranhão,2103000,"[[169, 501], [111, 169], None, None]","[111, 169, 501]",2,169,501,nan,1


In [66]:
df = df[['value_cum', 'value', 'ori_muni_name', 'ori_uf_name',
       'ori_co_ibge', 'des_muni_name', 'des_uf_name', 'des_co_ibge',
       'path_correct', 'path_len', 'muni_1', 'muni_2', 'muni_3', 'ones']]

In [67]:
df.to_csv('/Users/andreza/Downloads/FF_Manaus_hubs4.csv')

# Analise do que já concluiu

In [68]:
df_conclu = pd.concat(lis_dfs)

In [12]:
#df_conclu[df_conclu.des_muni_name == 'Simões Filho']

In [69]:
df_conclu.des_uf_name.unique()

array(['São Paulo', 'Amazonas', 'Minas Gerais', 'Maranhão', 'Ceará',
       'Paraná', 'Bahia', 'Alagoas', 'Pará', 'Espírito Santo',
       'Rio Grande do Norte', 'Piauí', 'Pernambuco', 'Rio Grande do Sul',
       'Santa Catarina', 'Paraíba', 'Goiás', 'Rio de Janeiro', 'Sergipe',
       'Tocantins', 'Mato Grosso', 'Mato Grosso do Sul', 'Roraima',
       'Rondônia', 'Acre', 'Amapá', 'Distrito Federal'], dtype=object)

In [70]:
for uf in df_conclu.des_uf_name.unique():

    print(uf, df_conclu[df_conclu.des_uf_name == uf].ori_co_ibge.nunique()) # 92 - mas parece que só roda 87

São Paulo 1
Amazonas 1
Minas Gerais 1
Maranhão 1
Ceará 1
Paraná 1
Bahia 1
Alagoas 1
Pará 1
Espírito Santo 1
Rio Grande do Norte 1
Piauí 1
Pernambuco 1
Rio Grande do Sul 1
Santa Catarina 1
Paraíba 1
Goiás 1
Rio de Janeiro 1
Sergipe 1
Tocantins 1
Mato Grosso 1
Mato Grosso do Sul 1
Roraima 1
Rondônia 1
Acre 1
Amapá 1
Distrito Federal 1


In [71]:
uf_select = df_conclu[df_conclu.des_uf_name == 'São Paulo']

In [72]:
uf_select.ori_co_ibge.nunique()

1

In [73]:
for value in uf_select.ori_co_ibge.unique():

    print(value, '-', uf_select[uf_select.ori_co_ibge == value].des_co_ibge.nunique())

1302603 - 190


In [74]:
set(df_conclu.ori_co_ibge.unique()) - set(df_conclu[df_conclu.des_uf_name == 'Bahia'].ori_co_ibge.unique())

set()

In [21]:
df_conclu[df_conclu.des_uf_name == 'Bahia'].des_co_ibge.nunique()

108